In [1]:
from tensorflow.keras.layers import Dropout, Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras import saving
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

@saving.register_keras_serializable()
class StandardModel(keras.Sequential):
    def __init__(self, input_shape=(32, 32, 3), **kwargs):
        super(StandardModel, self).__init__([
            layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Dropout(0.25),
            layers.Flatten(),
            layers.Dense(128, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(1, activation='softmax')
        ])



# Load CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Preprocess the data
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Assuming binary classification for simplicity, modify targets
y_train_bin = (y_train[:, 0] == 1).astype(int)  # Classify as class 0 or not
y_test_bin = (y_test[:, 0] == 1).astype(int)

# Initialize and train the model
model = StandardModel(input_shape=(32, 32, 3))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train_bin, epochs=1, batch_size=64)

# Save the model
model.save('my_model.keras')

# Load the model
loaded_model = load_model('my_model.keras', custom_objects={'StandardModel': StandardModel})

# Evaluate the loaded model
loss, accuracy = loaded_model.evaluate(x_test, y_test_bin)
print(f'Loss: {loss}, Accuracy: {accuracy}')


/Users/josephcarnec/Documents/CIFAR/.conda/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
/Users/josephcarnec/Documents/CIFAR/.conda/lib/python3.11/site-packages/keras/src/ops/nn.py:545: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


782/782 ━━━━━━━━━━━━━━━━━━━━ 30s 37ms/step - accuracy: 0.0994 - loss: 0.2572


ValueError: Sequential model 'standard_model_1' has already been configured to use input shape (None, 32, 32, 3). You cannot add a different Input layer to it.